Useful links
-----------
https://boto3.amazonaws.com/v1/documentation/api/latest/guide/migrationec2.html

In [1]:
import boto3
from boto3.resources.base import ServiceResource
from mypy_boto3_ec2 import EC2Client

region_name = 'eu-west-3'
key_name = 'aws_gth_keys'

In [2]:
node_session = boto3.Session(profile_name='ec2_node')
creator_session = boto3.Session(profile_name='ec2_creator')

ec2_resource_node: ServiceResource = node_session.resource('ec2', region_name=region_name)
ec2_client_node: EC2Client = node_session.client('ec2', region_name=region_name)
ec2_resource_creator: ServiceResource = creator_session.resource('ec2', region_name=region_name)
ec2_client_creator: EC2Client = creator_session.client('ec2', region_name=region_name)

ec2_client_creator, ec2_resource_creator, ec2_client_node, ec2_resource_node

(<botocore.client.EC2 at 0x7f9c96c3c340>,
 ec2.ServiceResource(),
 ec2.ServiceResource())

In [3]:
import os

def get_public_ip(ec2_client: EC2Client, instance_id):
    reservations = ec2_client.describe_instances(InstanceIds=[instance_id]).get("Reservations")

    for reservation in reservations:
        for instance in reservation['Instances']:
            print(instance.get("PublicIpAddress"))


def create_key_pair(ec2_client: EC2Client, key_pair_name: str) -> None:
    key_pair = ec2_client.create_key_pair(KeyName=key_pair_name)

    private_key = key_pair["KeyMaterial"]

    # write private key to file with 400 permissions
    with os.fdopen(os.open(f"/tmp/{key_pair_name}.pem", os.O_WRONLY | os.O_CREAT, 0o400), "w+") as handle:
        handle.write(private_key)

# create_key_pair(ec2_client, key_name)

In [4]:
from mypy_boto3_ec2.service_resource import Instance
from typing import List

instances: List[Instance] = ec2_resource_creator.create_instances(
    ImageId='ami-0f7559f51d3a22167',
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    KeyName=key_name,
    SecurityGroupIds=['sg-0ea7ad2589dc158bf'],
    IamInstanceProfile={
        'Arn': 'arn:aws:iam::747137879128:instance-profile/BlockchainNode',
      # 'Name': 'AmazonSSMRoleForInstancesQuickSetup'
    },
    # IamInstanceProfile={
    #     'Arn': 'arn:aws:iam::747137879128:instance-profile/AmazonSSMRoleForInstancesQuickSetup',
      # 'Name': 'MyBlockchainRole'
    # }
)
i: Instance = instances[0]
i.id, len(instances), type(instances[0]), i

('i-0859b7c76b25992c1',
 1,
 boto3.resources.factory.ec2.Instance,
 ec2.Instance(id='i-0859b7c76b25992c1'))

 BlockDeviceMappings, ImageId, InstanceType, Ipv6AddressCount, Ipv6Addresses, KernelId, KeyName, MaxCount, MinCount, Monitoring, Placement, RamdiskId, SecurityGroupIds, SecurityGroups, SubnetId, UserData, AdditionalInfo, ClientToken, DisableApiTermination, DryRun, EbsOptimized, IamInstanceProfile, InstanceInitiatedShutdownBehavior, NetworkInterfaces, PrivateIpAddress, ElasticGpuSpecification, ElasticInferenceAccelerators, TagSpecifications, LaunchTemplate, InstanceMarketOptions, CreditSpecification, CpuOptions, CapacityReservationSpecification, HibernationOptions, LicenseSpecifications, MetadataOptions, EnclaveOptions, PrivateDnsNameOptions, MaintenanceOptions, DisableApiStop

In [5]:
i.wait_until_running()
i.reload()
get_public_ip(ec2_client_creator, i.id), i.state

13.39.16.209


(None, {'Code': 16, 'Name': 'running'})

In [11]:
vars(i)

{'meta': ResourceMeta('ec2', identifiers=['id']), '_id': 'i-0859b7c76b25992c1'}

In [6]:
import time
# time.sleep(20)
instance_id = i.id

ssm_client = node_session.client('ssm', region_name=region_name)

while True:
    try:
        response = ssm_client.send_command(
            InstanceIds=[instance_id],
            DocumentName="AWS-RunShellScript",
            Parameters={'commands': ['ls -la /']}
        )
        break
    except Exception as e:
        print(e)
        time.sleep(3)
command_id = response['Command']['CommandId']
time.sleep(2)
output = ssm_client.get_command_invocation(CommandId=command_id, InstanceId=instance_id)
while output['Status'] == "InProgress":
    output = ssm_client.get_command_invocation(CommandId=command_id, InstanceId=instance_id)
print(output['StandardOutputContent'])


An error occurred (InvalidInstanceId) when calling the SendCommand operation: Instances [[i-0859b7c76b25992c1]] not in a valid state for account 747137879128
An error occurred (InvalidInstanceId) when calling the SendCommand operation: Instances [[i-0859b7c76b25992c1]] not in a valid state for account 747137879128
total 72
drwxr-xr-x  19 root root  4096 Jul 28 16:28 .
drwxr-xr-x  19 root root  4096 Jul 28 16:28 ..
lrwxrwxrwx   1 root root     7 Jun 10 10:44 bin -> usr/bin
drwxr-xr-x   4 root root  4096 Jun 10 10:50 boot
drwxr-xr-x  17 root root  3220 Jul 28 16:28 dev
drwxr-xr-x  95 root root  4096 Jul 28 16:28 etc
drwxr-xr-x   3 root root  4096 Jul 28 16:28 home
lrwxrwxrwx   1 root root     7 Jun 10 10:44 lib -> usr/lib
lrwxrwxrwx   1 root root     9 Jun 10 10:44 lib32 -> usr/lib32
lrwxrwxrwx   1 root root     9 Jun 10 10:44 lib64 -> usr/lib64
lrwxrwxrwx   1 root root    10 Jun 10 10:44 libx32 -> usr/libx32
drwx------   2 root root 16384 Jun 10 10:47 lost+found
drwxr-xr-x   2 root root

In [10]:
from typing import Iterator


def stop_instance(ec2_client, instance_id):
    response = ec2_client.stop_instances(InstanceIds=[instance_id])
    print(f'stop {instance_id=}: {response}')


def terminate_instance(ec2_client, instance_id):
    response = ec2_client.terminate_instances(InstanceIds=[instance_id])
    print(f'terminate {instance_id=}: {response}')


def running_instances_ids(ec2_client) -> Iterator[str]:
    reservations = ec2_client.describe_instances(Filters=[
        {
            "Name": "instance-state-name",
            "Values": ["running"],
        }
    ]).get("Reservations")

    for reservation in reservations:
        for instance in reservation["Instances"]:
            yield instance["InstanceId"]

def terminate_all_instances(ec2_client):
    for instance_id in running_instances_ids(ec2_client):
        terminate_instance(ec2_client, instance_id)


def get_running_instances(ec2_client, ec2_resource):
    reservations = ec2_client.describe_instances(Filters=[
        {
            "Name": "instance-state-name",
            "Values": ["running"],
        }
    ]).get("Reservations")

    for reservation in reservations:
        for instance in reservation["Instances"]:
            instance_id = instance["InstanceId"]
            instance_type = instance["InstanceType"]
            public_ip = instance["PublicIpAddress"]
            private_ip = instance["PrivateIpAddress"]
            print(f"{instance_id}, {instance_type}, {public_ip=}, {private_ip=}")

    for status in ec2_resource.meta.client.describe_instance_status()['InstanceStatuses']:
        print(status)

get_running_instances(ec2_client_creator, ec2_resource_creator)


In [8]:
terminate_all_instances(ec2_client_creator)

terminate instance_id='i-0859b7c76b25992c1': {'TerminatingInstances': [{'CurrentState': {'Code': 32, 'Name': 'shutting-down'}, 'InstanceId': 'i-0859b7c76b25992c1', 'PreviousState': {'Code': 16, 'Name': 'running'}}], 'ResponseMetadata': {'RequestId': '98c708dc-2b11-4cbf-a156-8d4a6758a992', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '98c708dc-2b11-4cbf-a156-8d4a6758a992', 'cache-control': 'no-cache, no-store', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'vary': 'accept-encoding', 'content-type': 'text/xml;charset=UTF-8', 'transfer-encoding': 'chunked', 'date': 'Thu, 28 Jul 2022 16:28:41 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}


In [9]:
response = ec2_client_creator.terminate_instances(
    InstanceIds=[
        instance.id for instance in instances
    ],
)